In [ ]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
os.chdir("/Users/filippo/Developer/tesi")

In [ ]:
try:
    del df
except:
    pass
df = None
df_files = pd.DataFrame(columns=['files','tissue', 'status'])
for tissuetype in os.listdir("merged/data"):
    if ".gz" not in tissuetype:
        continue
    print(tissuetype)
    newdf = pd.read_csv("merged/data/%s"%tissuetype, index_col=[0], header=0, sep='\t').drop('Entrez_Gene_Id', axis=1)
    if df is None:
        df=newdf
    else:
        df = df.join(newdf, )
    newfiles = {
        'files': newdf.columns.values,
        'tissue':[tissuetype for _ in newdf.columns],
        'status': ['healthy' if 'gtex' in tissuetype else 'tumour' for _ in newdf.columns]
    }
    df_files = df_files.append(pd.DataFrame(data=newfiles))
    del newdf
df_files.set_index('files',inplace=True)

In [ ]:
symb_to_ensg = pd.read_csv("https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=md_ensembl_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_hgnc_id&format=text&submit=submit", sep='\t', index_col=0).dropna()

In [ ]:
df_files.to_csv("merged/files.dat", index=True, header=True)

In [ ]:
remapper = symb_to_ensg[symb_to_ensg.index.isin(df.index.values)].to_dict()['Ensembl ID(supplied by Ensembl)']

In [ ]:
df = df.loc[remapper.keys(),:]
df = df.rename(index=remapper)
df.reset_index(inplace=True)
df.drop_duplicates(subset='Hugo_Symbol', keep='last', inplace=True)
df.set_index('Hugo_Symbol',inplace=True)

In [ ]:
df.dropna().to_csv("merged/mainTable.csv", index=True, header=True)

In [ ]:
#tf.reset_default_graph()

In [ ]:
tfdf = tf.convert_to_tensor(pd.read_csv("merged/mainTable.csv", header=0, index_col=0))

In [ ]:
def make_zipf(df):
    return tf.sort(tf.math.divide(tf.math.reduce_sum(df, 1),tf.math.reduce_sum(df)), direction='DESCENDING')

In [ ]:
with tf.name_scope('tacos') as scope:
    m = tf.math.reduce_mean(tfdf,1, name='means')
    v = tf.math.reduce_variance(tfdf, 1)
    two = tf.dtypes.cast(tf.constant(2),tf.float64)
    cv2 = tf.math.divide(v, tf.math.pow(m,two))
    occ = tf.map_fn(lambda x: tf.count_nonzero(x, dtype=tf.float64),tfdf)
    M = tf.reduce_sum(tfdf, 0)
    zipf = make_zipf(tfdf)

In [ ]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("output", sess.graph)
    runned = sess.run([m,v,cv2, occ, zipf,M])
    nulltab = []
    for Mi in runned[5]:
        nulltab.append(np.random.multinomial(Mi,runned[4]))
    tfdf_null=tf.transpose(tf.stack(nulltab))
    zipf_null = tf.sort(tf.math.divide(tf.math.reduce_sum(tfdf_null, 1),tf.math.reduce_sum(tfdf_null)), direction='DESCENDING')
    runned_null = sess.run([zipf_null, tf.math.reduce_mean(tfdf_null,1), tf.math.reduce_variance(tfdf_null, 1)])
    writer.close()
    sess.close()

In [ ]:
plt.plot(runned[4])
plt.plot(runned_null[0], c='r', ls='--')
#plt.scatter(runned[3],runned[0])
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
plt.scatter(runned[0], runned[1])
plt.scatter(runned_null[1], runned_null[2])
plt.plot(np.logspace(-1,5),np.power(np.logspace(-1,5),1))
plt.plot(np.logspace(-1,5),np.power(np.logspace(-1,5),2))
plt.xscale('log')
plt.yscale('log')
plt.show()